In [1]:
import torch
from collections import Counter


In [2]:
def unit_step_with_rand(x, previous_value):
    return 1 - previous_value if x <= torch.rand(1) else previous_value

def dwell_function(tensor, tao):
    return tao * torch.exp(tensor)

def other_transform(tensor: torch.Tensor, delta, previous_value):
    return unit_step_with_rand(1 - torch.exp(-delta/dwell_function(tensor, 10**(-11))), previous_value)

def number_from_tensor(tensor: torch.Tensor):
    powers_of_two = torch.vander(torch.Tensor([2]), N=tensor.size(dim=0)+1, increasing=True)
    scalar_mul = tensor * torch.reshape(powers_of_two, (-1,))[1:]
    return 1 + scalar_mul.sum()

In [3]:
class MyIntegerFactorizationModel:

    def __init__(self, F: int, fitting_parameter: float, delta: float):
        self.F = F
        length = F.bit_length()
        self.P = (length - 1) // 2
        self.Q = length - 2 - self.P
        #self.eternal_tensor = torch.ones(self.P + self.Q, requires_grad=True)
        self.eternal_tensor = torch.randint(0, 2, size=(self.P + self.Q,), dtype=torch.float32, requires_grad=True)
        self.tensor_collector = Counter()
        self.fitting_parameter = fitting_parameter
        self.random_vector = torch.rand(self.P + self.Q)
        self.delta = delta

    def energy_function(self, tensor: torch.Tensor) -> torch.Tensor:
        return self.fitting_parameter * (number_from_tensor(tensor[:self.P]) * number_from_tensor(tensor[self.P:]) - self.F) ** 2

    def calculate_gradient(self):
        energy = self.energy_function(self.eternal_tensor)
        energy.backward(torch.ones(energy.shape))
        self.eternal_tensor.retain_grad()
        return self.eternal_tensor.grad


    def evaluate(self):
        self.manage_counting()
        gradient = self.calculate_gradient()
        index = torch.randint(self.P + self.Q, (1, ))
        current_grad = gradient[index]
        previous_value = self.eternal_tensor[index]
        trans = other_transform(-current_grad if self.eternal_tensor[index] == 0 else current_grad, self.delta, previous_value)
        with torch.no_grad():
            self.eternal_tensor.data[index] = trans

    def manage_counting(self):
        first_number = number_from_tensor(self.eternal_tensor[:self.P])
        second_number = number_from_tensor(self.eternal_tensor[self.P:])
        if first_number != 1:
            self.tensor_collector[first_number.item()] += 1
        if second_number != 1:
            self.tensor_collector[second_number.item()] += 1

In [7]:
integer_factorization = MyIntegerFactorizationModel(F=57, fitting_parameter=5, delta = 1)

for i in range(2000):
    integer_factorization.evaluate()

print(integer_factorization.tensor_collector)

Counter({7.0: 3995, 3.0: 4, 5.0: 1})
